# Importowanie bibliotek
W projekcie wykorzystano następujące biblioteki:
- TensorFlow i Keras: Do budowy i trenowania modelu głębokiego uczenia.
- NumPy i Pandas: Do manipulacji danymi i ich wstępnego przetwarzania.
- Scikit-learn: Do skalowania danych oraz podziału na zbiory treningowe i testowe.
- Keras Tuner: Do optymalizacji hiperparametrów modelu.
- Callbacks w Keras: EarlyStopping i ReduceLROnPlateau zapewniają zatrzymanie treningu w odpowiednim momencie i dostosowywanie tempa uczenia.

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras_tuner.tuners import BayesianOptimization
from tensorflow.keras.layers import Input

# Przygotowanie danych
- Wczytanie danych: Dane zostały zaimportowane z pliku CSV.
- Wybór cech: Wybrano kolumny numeryczne: ['Size(L)', 'OG', 'FG', 'IBU', 'ABV', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency'].
- Uzupełnienie braków: Wartości brakujące w danych zastąpiono średnią z kolumn.
- Normalizacja: Dane zostały znormalizowane przy użyciu StandardScaler, co pozwala na lepszą konwergencję modelu.
- Podział danych: Dane podzielono na zbiór treningowy (80%) i testowy (20%) z losowym seedem 42.

In [3]:
dane = pd.read_csv('../data.csv')

kolumny_numeryczne = ['Size(L)', 'OG', 'FG', 'IBU', 'ABV', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
X = dane[kolumny_numeryczne].copy()

X = X.fillna(X.mean())

y = dane['Color'].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Struktura modelu
Model został zbudowany w formie sekwencyjnej.

Warstwy:
- Warstwa wejściowa: 288 neuronów z aktywacją ReLU.
- Cztery warstwy ukryte:
  - Każda warstwa ma regulowaną liczbę neuronów (od 32 do 512, w krokach po 32) oraz Dropout (od 0.0 do 0.5).
  - Każda warstwa jest wsparta Batch Normalization, stabilizującą proces uczenia.
- Warstwa wyjściowa: Jeden neuron do przewidywania koloru piwa (regresja).

Regularyzacja: Dropout zapobiega przeuczeniu, a Batch Normalization przyspiesza i stabilizuje proces uczenia.

Kompilacja modelu
- Optymalizator: Adam z dynamiczną szybkością uczenia (od 1e-4 do 1e-2, logarytmiczny zakres).
- Funkcja straty: Mean Squared Error (MSE).
- Metryka: Mean Absolute Error (MAE).

In [4]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(
        units=hp.Int('units_input', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_input', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(
        units=hp.Int('units_1', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(
        units=hp.Int('units_2', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(
        units=hp.Int('units_3', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(1))
    
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='mean_squared_error',
        metrics=['mean_absolute_error']
    )
    
    return model


# Proces optymalizacji hiperparametrów
Keras Tuner:
- Rodzaj tunera: BayesianOptimization.
- Cel: Minimalizacja val_mean_absolute_error.
- Liczba prób: 20.
- Callbacki: EarlyStopping (patience=5) i TensorBoard do wizualizacji.

Wynik:
- Najlepsze val_mean_absolute_error: 7.396.
- Łączny czas optymalizacji: ~15 minut.

In [5]:
tuner = BayesianOptimization(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=20,
    directory='logs/fit',
    project_name='beer_color_prediction')

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir='logs/fit/beer_color_prediction')

tuner.search(X_train, y_train, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[early_stopping, tensorboard_callback],
             batch_size=32)

Trial 20 Complete [00h 00m 40s]
val_mean_absolute_error: 7.641167640686035

Best val_mean_absolute_error So Far: 7.396204948425293
Total elapsed time: 00h 14m 51s


In [6]:
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 288)            │         2,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 288)            │        83,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 384)            │       110,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 384)            │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │        12,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 213,409 (833.63 KB)

 Trainable params: 211,425 (825.88 KB)

 Non-trainable params: 1,984 (7.75 KB)

# Wyniki najlepszego modelu
Architektura modelu:
- Liczba parametrów: 213,409.
- Trainable: 211,425 (pozostałe to parametry Batch Normalization).

Proces treningu:
- Liczba epok: 20.
- Batch size: 32.
- Final Validation MAE: 7.81.

In [7]:
history = best_model.fit(X_train, y_train, 
                         epochs=20, 
                         validation_split=0.2, 
                         callbacks=[early_stopping, tensorboard_callback],
                         batch_size=32)

best_model.summary()

final_val_mae = history.history['val_mean_absolute_error'][-1]
print(f"Final Validation MAE: {final_val_mae:.2f}")

Epoch 1/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 116.5976 - mean_absolute_error: 7.9712 - val_loss: 116.0205 - val_mean_absolute_error: 8.0611
Epoch 2/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 117.9371 - mean_absolute_error: 8.0665 - val_loss: 115.9022 - val_mean_absolute_error: 7.5477
Epoch 3/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 114.8725 - mean_absolute_error: 7.9188 - val_loss: 111.3817 - val_mean_absolute_error: 7.7977
Epoch 4/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 115.9947 - mean_absolute_error: 7.9555 - val_loss: 113.4200 - val_mean_absolute_error: 7.9574
Epoch 5/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 117.6537 - mean_absolute_error: 8.0534 - val_loss: 114.1915 - val_mean_absolute_error: 7.8140
Epoch 6/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 117.0835 - mean_absolute_error: 8.0180 - val_loss: 112.1130 - val_mean_absolute_error: 7.8873
Epoch 7/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 11

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 288)            │         2,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 288)            │        83,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 384)            │       110,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 384)            │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │        12,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 636,261 (2.43 MB)

 Trainable params: 211,425 (825.88 KB)

 Non-trainable params: 1,984 (7.75 KB)

 Optimizer params: 422,852 (1.61 MB)

Final Validation MAE: 7.81


In [15]:
%reload_ext tensorboard
%tensorboard --logdir logs/fit/beer_color_prediction

Reusing TensorBoard on port 6008 (pid 26608), started 0:00:34 ago. (Use '!kill 26608' to kill it.)